In [1]:
#yes we boutta make a classifier to detect a language, cuz why not??
#might test later
#libraries
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import text
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import random as rd
import os
from scipy import stats as st
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

def paragraphsplitter(path, minsize=40):
    with open(path, "r", encoding="utf8") as file:
        paragraphs=[para for para in file.read().split("\n\n") if len(para)>minsize]
    return paragraphs

def dictsplitter(path, repeatingwords=None):
    words=[]
    with open(path, "r", encoding="utf8") as file:
        if path=="Files/languageproject/DICT/Dictionaries/AR.dic":
            for word in file.read().split("\n"):
                if "/" in word:
                    words.append(word[word.index("/")+1:])
                else:
                    words.append(word)
        else:
            for word in file.read().split("\n"):
                if "/" in word:
                    words.append(word[:word.index("/")])
                else:
                    words.append(word)
        if type(repeatingwords)==int:
            words+=str(words)*repeatingwords
            return words
        else:
            return words

#main here
    #preparing data
language={"AA":"Afar",
            "AB":"Abkhazian",
            "AE":"Avestan",
            "AF":"Afrikaans",
            "AK":"Akan",
            "AM":"Amharic",
            "AN":"Aragonese",
            "AR":"Arabic",
            "AS":"Assamese",
            "AV":"Avaric",
            "AY":"Aymara",
            "AZ":"Azerbaijani",
            "BA":"Bashkir",
            "BE":"Belarusian",
            "BG":"Bulgarian",
            "BH":"Bihari",
            "BI":"Bislama",
            "BM":"Bambara",
            "BN":"Bengali",
            "BO":"Tibetan",
            "BR":"Breton",
            "BS":"Bosnian",
            "CA":"Catalan",
            "CE":"Chechen",
            "CH":"Chamorro",
            "CO":"Corsican",
            "CR":"Cree",
            "CS":"Czech",
            "CU":"Church",
            "CV":"Chuvash",
            "CY":"Welsh",
            "DA":"Danish",
            "DE":"German",
            "DV":"Divehi",
            "DZ":"Dzongkha",
            "EE":"Ewe",
            "EL":"Greek",
            "EN":"English",
            "EO":"Esperanto",
            "ES":"Spanish",
            "ET":"Estonian",
            "EU":"Basque",
            "FA":"Persian",
            "FF":"Fulah",
            "FI":"Finnish",
            "FJ":"Fijian",
            "FO":"Faroese",
            "FR":"French",
            "GA":"Irish",
            "GD":"Gaelic",
            "GL":"Galician",
            "GN":"Guarani",
            "GU":"Gujarati",
            "GV":"Manx",
            "HA":"Hausa",
            "HE":"Hebrew",
            "HI":"Hindi",
            "HO":"HiriMotu",
            "HR":"Croatian",
            "HT":"Haitian",
            "HU":"Hungarian",
            "HY":"Armenian",
            "HZ":"Herero",
            "IA":"Interlingua",
            "ID":"Indonesian",
            "IE":"Interlingue",
            "IG":"Igbo",
            "II":"SichuanYi",
            "IK":"Inupiaq",
            "IO":"Ido",
            "IS":"Icelandic",
            "IT":"Italian",
            "IU":"Inuktitut",
            "JA":"Japanese",
            "JV":"Javanese",
            "KA":"Georgian",
            "KG":"Kongo",
            "KI":"Kikuyu",
            "KJ":"Kuanyama",
            "KK":"Kazakh",
            "KL":"Kalaallisut",
            "KM":"CentralKhmer",
            "KN":"Kannada",
            "KO":"Korean",
            "KR":"Kanuri",
            "KS":"Kashmiri",
            "KU":"Kurdish",
            "KV":"Komi",
            "KW":"Cornish",
            "KY":"Kirghiz;Kyrgyz",
            "LA":"Latin",
            "LB":"Luxembourgish",
            "LG":"Ganda",
            "LI":"Limburgan",
            "LN":"Lingala",
            "LO":"Lao",
            "LT":"Lithuanian",
            "LU":"Luba-Katanga",
            "LV":"Latvian",
            "MG":"Malagasy",
            "MH":"Marshallese",
            "MI":"Maori",
            "MK":"Macedonian",
            "ML":"Malayalam",
            "MN":"Mongolian",
            "MR":"Marathi",
            "MS":"Malay",
            "MT":"Maltese",
            "MY":"Burmese",
            "NA":"Nauru",
            "NE":"Nepali",
            "NG":"Ndonga",
            "NL":"Dutch",
            "NN":"NorwegianNynorsk",
            "NO":"Norwegian",
            "NR":"Ndebele",
            "NV":"Navajo",
            "NY":"Chichewa",
            "OJ":"Ojibwa",
            "OM":"Oromo",
            "OR":"Oriya",
            "OS":"Ossetian",
            "PA":"Panjabi",
            "PI":"Pali",
            "PL":"Polish",
            "PS":"Pushto",
            "PT":"Portuguese",
            "QU":"Quechua",
            "RM":"Romansh",
            "RN":"Rundi",
            "RO":"Romanian",
            "RU":"Russian",
            "RW":"Kinyarwanda",
            "SA":"Sanskrit",
            "SC":"Sardinian",
            "SD":"Sindhi",
            "SE":"NorthernSami",
            "SG":"Sango",
            "SI":"Sinhala",
            "SK":"Slovak",
            "SL":"Slovenian",
            "SM":"Samoan",
            "SN":"Shona",
            "SO":"Somali",
            "SQ":"Albanian",
            "SR":"Serbian",
            "SS":"Swati",
            "ST":"Sotho",
            "SU":"Sundanese",
            "SV":"Swedish",
            "SW":"Swahili",
            "TA":"Tamil",
            "TE":"Telugu",
            "TG":"Tajik",
            "TH":"Thai",
            "TI":"Tigrinya",
            "TK":"Turkmen",
            "TL":"Tagalog",
            "TN":"Tswana",
            "TO":"Tonga",
            "TR":"Turkish",
            "TS":"Tsonga",
            "TT":"Tatar",
            "TW":"Twi",
            "TY":"Tahitian",
            "UG":"Uighur",
            "UK":"Ukrainian",
            "UR":"Urdu",
            "UZ":"Uzbek",
            "VE":"Venda",
            "VI":"Vietnamese",
            "VO":"Volapük",
            "WA":"Walloon",
            "WO":"Wolof",
            "XH":"Xhosa",
            "YI":"Yiddish",
            "YO":"Yoruba",
            "ZA":"Zhuang",
            "ZH":"Chinese",
            "ZU":"Zulu"}                                              #important for label later

data=[];label=[]
folerpath="Files/languageproject/"
dictlocation=os.listdir(folerpath)
files=[name for name in dictlocation if name.endswith(".txt")]         #location and name of file, might use endswith later

for file in files:
    path=folerpath+file
    paras=paragraphsplitter(path, 25)
    data.extend(paras)
    label+=[file[:2]]*len(paras)
    
#Yk what fuck this, imma just use dictionary instead!   credit: https://github.com/titoBouzout/Dictionaries/tree/master
folerpath="Files/languageproject/DICT/Dictionaries/"
dictlocation=os.listdir(folerpath)
files=[name for name in dictlocation if name.endswith(".dic")]
for file in files:
    print("*", end="")
    path=folerpath+file
    words=dictsplitter(path, repeatingwords=1)
    data.extend(words)
    label+=[file[:2]]*len(words)
print(f'\nDict loaded')

#randomize
merged=list(zip(data, label))
merged=rd.sample(merged, len(merged))
data, label = list(zip(*merged))
print("data randomized")

#split train test
train_data, test_data, train_label, test_label=train_test_split(data, label, train_size= 0.8)
print("data divided sucsessfully")

#convert to array
vectorizer=text.CountVectorizer()
train_data=vectorizer.fit_transform(train_data)
test_data=vectorizer.transform(test_data)
print("data converted to array")

#model of mutinomial
model=MultinomialNB(alpha=0.05)
model.fit(train_data, train_label)
print("model fitted ")

predicted=model.predict(test_data)

print(f'Accuracy Score :{accuracy_score(test_label, predicted)}')
print(f'F1_Score       :{f1_score(test_label, predicted, average="macro")}')
#print(confusion_matrix(test_label, predicted))                                  #How accurate the model is

***************************************************
Dict loaded


In [ ]:
def predictingyourlanguagecuh(sentence):
    yoursentence=vectorizer.transform(sentence)
    predict=model.predict(yoursentence)
    yourlanguageis=language[predict[0]]
    print(f'predicted language:     {yourlanguageis}')

dummy=""
predictingyourlanguagecuh([""])